In [1]:
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [3]:
import pickle

In [4]:
with open('body_pose_estimate.pkl', 'rb') as f:
    model = pickle.load(f)

In [9]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.6, min_tracking_confidence=0.6) as holistic:
    while cap.isOpened():
        ret, frame =  cap.read()
        
        #Recoloring to RGB and set flag writable to False
        image =  cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        #Detecting landmarks
        results = holistic.process(image)
        
        #Recoloring back to BGR and set flag writable to True
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
         
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
#         # 2. Right hand
#         mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
#                                  mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
#                                  mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
#                                  )

#         # 3. Left Hand
#         mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
#                                  mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
#                                  mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
#                                  )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        try:
            pose = results.pose_landmarks.landmark
            pose_landmarks =  list(np.array([[i.x,i.y,i.z,i.visibility] for i in pose]).flatten())
            
            face = results.face_landmarks.landmark
            face_landmarks =  list(np.array([[i.x,i.y,i.z,i.visibility] for i in face]).flatten())
            
            total_landmarks = pose_landmarks + face_landmarks
            
            
                  

            X = pd.DataFrame([total_landmarks])
            output =  model.predict(X)
            
            
    

            cv2.putText(image,'CLASS', (95,12), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, output[0] ,(90,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
  
        
            
            
            
        except Exception as e:
            print(e)
            
            
        cv2.imshow("Detection",image)
        
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

            
cap.release()
cv2.destroyAllWindows()